# Datová analýza exoplanetárního datasetu</h1>
## Semestrální práce z předmětu 4ST101
### *Autor: Jakub Vítovec*

## 1. Charakteristika zpracovávaných dat.
Ve své semestrální práci se budu zabývat daty získanými z veřejné databáze [Keggle](https://www.kaggle.com/). Konkrétní dataset je dostupný na URL [https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system](https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system) a byl vytvořen uživatelem Sathyanarayan Rao z dat publikovaných vesmírnou agenturou NASA. Celkem se v datasetu nachází 32552 různých dosud objevených exoplanet a základní data o nich.
Exoplaneta je planeta, která obíhá okolo jiné hvězdy, než okolo Slunce. První exoplaneta byla objevena v roce 1998 a počet jejich počet stále roste.

### Čištění dat
V tomto oddíle vyčistíme data od nežádoucích hodnot.

In [7]:
import numpy as np;
import pandas as pd;

dataset = pd.read_csv("./exoplanets_dataset.csv");
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32552 entries, 0 to 32551
Data columns (total 93 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Unnamed: 0                               32552 non-null  int64  
 1   Planet Name                              32552 non-null  object 
 2   Host Name                                32552 non-null  object 
 3   Number of Stars                          32552 non-null  int64  
 4   Number of Planets                        32552 non-null  int64  
 5   Discovery Method                         32552 non-null  object 
 6   Discovery Year                           32552 non-null  int64  
 7   Discovery Facility                       32552 non-null  object 
 8   Orbital Period [days]                    29747 non-null  float64
 9    Planet Radius [Earth Radius]            22711 non-null  float64
 10  Planet Mass or Mass*sin(i) [Earth Mass]  4699 

Jak vidíme dataset obsahuje značné množství nepojmenovaných sloupců - Unnamed. Ty z datasetu odstraníme jako první.

In [16]:
for column_name in dataset.columns:
    if ("Unnamed" in column_name):
        del dataset[column_name]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32552 entries, 0 to 32551
Data columns (total 11 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Planet Name                              32552 non-null  object 
 1   Host Name                                32552 non-null  object 
 2   Number of Stars                          32552 non-null  int64  
 3   Number of Planets                        32552 non-null  int64  
 4   Discovery Method                         32552 non-null  object 
 5   Discovery Year                           32552 non-null  int64  
 6   Discovery Facility                       32552 non-null  object 
 7   Orbital Period [days]                    29747 non-null  float64
 8    Planet Radius [Earth Radius]            22711 non-null  float64
 9   Planet Mass or Mass*sin(i) [Earth Mass]  4699 non-null   float64
 10   Eccentricity                            16650